# Java PDFBox Demo: Santa Prime Paths

Quick demo of compiling Java from a Notebook, and running it using Jar files (libraries) from Kaggle dataset.

This renders a solution file for [Traveling Santa 2018 - Prime Paths][1] into a PDF file for a very high resolution view. You can zoom in on screen or create a large printout that might make nice artwork ;)

The Java source included below was [shared on the forums here][2], with some example PDFs.

[1]: https://www.kaggle.com/c/traveling-santa-2018-prime-paths
[2]: https://www.kaggle.com/c/traveling-santa-2018-prime-paths/discussion/76912

In [1]:
from IPython.display import IFrame
from glob import glob

In [2]:
%%writefile RenderSantaTripToPDF.java
import java.awt.Color;
import java.io.BufferedReader;
import java.io.File;
import java.io.FileReader;
import java.io.IOException;
import java.io.PrintStream;
import java.util.ArrayList;

import org.apache.pdfbox.pdmodel.PDDocument;
import org.apache.pdfbox.pdmodel.PDPage;
import org.apache.pdfbox.pdmodel.PDPageContentStream;
import org.apache.pdfbox.pdmodel.common.PDRectangle;

public class RenderSantaTripToPDF {

	public static void main(String[] args) throws Exception {

		if (args.length < 2) {
			System.out.println("Usage: java RenderSantaTripToPDF cities.csv solution.csv [output.pdf] [-c]");
			return;
		}

		final String cities_csv = args[0];
		final String sol_csv = args[1];
		final String out_pdf = (args.length > 2 ? args[2] : null);
		final boolean hsvColor = (args.length > 3 && args[3].startsWith("-c"));

		RenderSantaTripToPDF s = new RenderSantaTripToPDF();
		s.readCities(cities_csv);
		s.drawSolution(sol_csv, hsvColor);
		s.saveAndClose(out_pdf);
		s.printStats(System.out);
	}

	static boolean isPrime(final int n) {
		// check if n is a multiple of 2
		if (n % 2 == 0)
			return false;
		// if not, then just check the odds
		for (int i = 3; i * i <= n; i += 2) {
			if (n % i == 0)
				return false;
		}
		return true;
	}

	ArrayList<double[]> cities = new ArrayList<double[]>();
	PDDocument doc;
	PDPage page;
	PDPageContentStream contentStream;

	// from cities.csv
	final float maxX = 5100;
	final float maxY = 3400;
	// settings to tweak
	float border = 50;
	float lineWidth = 1.5f;
	float startMarkerSize = 5f; // diamond marking start of path, size in original units
	// computed score stats
	int nprime;
	int npenalised;
	double sum;
	double penalty;

	RenderSantaTripToPDF() throws Exception {
		doc = new PDDocument();
		float expand = 2 * border;
		final PDRectangle size = new PDRectangle(maxX + expand, maxY + expand);
		page = new PDPage(size); // replace PDPage.PAGE_SIZE_A4
		doc.addPage(page);
		contentStream = new PDPageContentStream(doc, page, true, true);
		contentStream.setLineWidth(lineWidth);
	}

	void saveAndClose(String out_pdf) throws Exception {
		contentStream.close();
		doc.save(pdfName(out_pdf));
		doc.close();
	}

	String pdfName(String out_pdf) {
		return out_pdf != null ? out_pdf : String.format("%.2f.pdf", sum + penalty);
	}

	void printStats(PrintStream out) {
		out.println("TSP score       : " + sum);
		out.println("Prime penalty   : " + penalty);
		out.println("10ths prime     : " + nprime);
		out.println("10ths penalised : " + npenalised);
		out.println("FINAL SCORE     : " + (sum + penalty));
	}

	void readCities(String cities_csv) throws IOException {

		BufferedReader r = new BufferedReader(new FileReader(new File(cities_csv)));
		r.readLine(); // header
		String line;

		while ((line = r.readLine()) != null) {
			final String[] p = line.split(",");
			final double x = Double.parseDouble(p[1]);
			final double y = Double.parseDouble(p[2]);
			cities.add(new double[] { x, y }); // use original coordinates in the page
		}
		r.close();
	}

	float pageX(double[] c) {
		return (float) c[0] + border;
	}

	float pageY(double[] c) {
		return (float) c[1] + border;
	}

	static double distance(double[] a, double[] b) {
		double dx = a[0] - b[0];
		double dy = a[1] - b[1];
		return Math.sqrt((dx * dx) + (dy * dy));
	}

	void drawSolution(String sol_csv, boolean hsvColor) throws Exception {

		BufferedReader r = new BufferedReader(new FileReader(new File(sol_csv)));
		r.readLine(); // header
		r.readLine(); // city 0

		double[] depot = cities.get(0);

		contentStream.setNonStrokingColor(Color.RED);
		float[] xs = { pageX(depot) - startMarkerSize, pageX(depot), pageX(depot) + startMarkerSize, pageX(depot) };
		float[] ys = { pageY(depot), pageY(depot) - startMarkerSize, pageY(depot), pageY(depot) + startMarkerSize };
		contentStream.fillPolygon(xs, ys);

		String line;
		int ind = 1;
		int lastCity = 0;
		double[] last = cities.get(lastCity);
		while ((line = r.readLine()) != null) {
			final int city = Integer.parseInt(line);
			double[] curr = cities.get(city);
			double link_score = distance(curr, last);
			sum += link_score;

			Color c = Color.black;
			if ((ind % 10) == 0) {
				if (isPrime(lastCity)) {
					c = Color.blue;
					nprime++;
				}
				else {
					c = Color.red;
					npenalised++;
					penalty += link_score * 0.1;
				}
			}

			if (hsvColor) {
				// simple alternate color scheme
				c = new Color(Color.HSBtoRGB(ind / (float) cities.size(), 1f, 1f));
			}

			contentStream.setStrokingColor(c);
			contentStream.drawLine(pageX(last), pageY(last), pageX(curr), pageY(curr));

			ind++;
			last = curr;
			lastCity = city;
		}
		r.close();
	}
}

In [3]:
cpath = '../input/java-jars/pdfbox-app-2.0.13.jar'
cities = '../input/traveling-santa-2018-prime-paths/cities.csv'

Compile the source

In [4]:
!javac -cp {cpath} RenderSantaTripToPDF.java

We now have a class file

In [5]:
!ls -l

Run it

In [6]:
!java -cp .:{cpath} RenderSantaTripToPDF {cities} ../input/pmtest1/submission.csv

In [7]:
!ls -l

This renders in the interactive session but not in the committed Notebook...

The PDF is available from the output files section &larr;

In [8]:
IFrame("1514325.09.pdf", width=900, height=650)

TODO: (maybe) add all public Notebook solutions and render them all in a loop.

In [9]:
glob('../input/**/*.csv', recursive=True)